### LIBRARIES ###

In [1]:
import os
import mne
import pandas as pd
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report, get_entities_from_fname
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import bids
from mne.datasets.eegbci import standardize
from fooof.utils import trim_spectrum
#For plots outside jupiter
get_ipython().run_line_magic("matplotlib", " qt ")

#### DATASETS

##### Medellin

In [2]:
antioquia={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'O1', 'OZ', 'O2'],
    'path':"D:/EEGs/PhD_datasets/BIDS/antioquia/"
}

#Path of the BIDS folder
bids_root = "D:/EEGs/PhD_datasets/BIDS/antioquia/"
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=antioquia #SI SE CAMBIA POR CALIFORNIA, SÍ SIRVE LA LINEA DE ABAJO

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

72 ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datas

FOR STANDARD AVERAGED EPOCHS APPROACH - DF WAS CALCULATED OVER THE MEAN PSD VECTOR, PSD VARIABILITY WAS CALCULATED FROM THE EPOCHS PSD VECTORS IN R

In [12]:
%%capture
%%time
#eegs = ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif']
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(44)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c    
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = 500
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'medellin'
        features['subject'] = subject
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
df = pd.DataFrame(dict_list)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/antioquia/derivatives/sovaharmony/df_medellin_5s_averaged.csv', index=False)

#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
# Loading epochs data

# df2 = pd.read_csv('D:/escritorio/FDA/medellin/df_medellin_5s_no_overlapping.csv')
# df2_filtered_epochs = df2[df2['epochs'] <= 28]
# df2_filtered_epochs['dfv_ch'] = np.std(df2_filtered_epochs.groupby(['subject','channel'])['df_ep'])
# 


In [ ]:
%%time
%%capture
dominant = []
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    print('BANDPASS FILTER 4 - 15 Hz & DOWNSAMPLE DONE')
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average='mean') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'medellin'
            features['subject'] = subject
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df = pd.DataFrame(dict_list)
            df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
df['dfv_ch'] = df['dfv_ch'].fillna(0)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/antioquia/derivatives/sovaharmony/df_medellin_5s_no_overlapping.csv', index=False)
print('CSV CREATED')

In [ ]:
%%time
%%capture
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    print('BANDPASS FILTER 4 - 15 Hz & DOWNSAMPLE DONE')
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        srate = downsample #This gets the sampling rate of the EEGLAB object
        nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
        freqs, psd = signal.welch(epochs._data[:,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average='mean') #This calculates PSD Welch
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features = {}
        features['center'] = 'medellin'
        features['subject'] = subject
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
        df = pd.DataFrame(dict_list)
        df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/antioquia/derivatives/sovaharmony/df_medellin_5s_averaged.csv', index=False)
print('CSV CREATED')

##### California #####

In [13]:
california={
'layout':{'extension':'.fif', 'session':['hc', 'on'], 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'PZ', 'PO3', 'O1', 'OZ', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'FP2', 'FZ', 'CZ'],
    'path':"D:/EEGs/PhD_datasets/BIDS/california/"
}

#Path of the BIDS folder
bids_root = "D:/EEGs/PhD_datasets/BIDS/california/"
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=california #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

31 ['D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc1\\ses-hc\\eeg\\sub-hc1_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc2\\ses-hc\\eeg\\sub-hc2_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc4\\ses-hc\\eeg\\sub-hc4_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc7\\ses-hc\\eeg\\sub-hc7_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc8\\ses-hc\\eeg\\sub-hc8_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc10\\ses-hc\\eeg\\sub-hc10_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\california\\derivatives\\sovaharmony\\sub-hc18\\ses-hc\\eeg\\sub-hc18_ses-hc_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\B

In [ ]:
%%time
%%capture
dominant = []
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(28)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c    
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        srate = downsample #This gets the sampling rate of the EEGLAB object
        nfft, noverlap, nperseg = int(2*srate), int(srate), int(2*srate)
        freqs, psd = signal.welch(epochs._data[:,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum', average = 'mean') #This calculates PSD Welch
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features = {}
        features['center'] = 'california'
        features['subject'] = subject
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
df = pd.DataFrame(dict_list)
#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
# Loading epochs data
df.to_csv('D:/EEGs/PhD_datasets/BIDS/california/derivatives/sovaharmony/df_california_5s_no_overlapping.csv', index=False)

# df2 = pd.read_csv('D:/escritorio/FDA/medellin/df_medellin_5s_no_overlapping.csv')
# df2_filtered_epochs = df2[df2['epochs'] <= 28]
# df2_filtered_epochs['dfv_ch'] = np.std(df2_filtered_epochs.groupby(['subject','channel'])['df_ep'])

In [14]:
%%capture
%%time
#eegs = ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif']
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(27)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c    
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = 500
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'california'
        features['subject'] = subject
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
df = pd.DataFrame(dict_list)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/california/derivatives/sovaharmony/df_california_5s_averaged.csv', index=False)

# df2 = pd.read_csv('D:/escritorio/FDA/medellin/df_medellin_5s_no_overlapping.csv')
# df2_filtered_epochs = df2[df2['epochs'] <= 28]
# df2_filtered_epochs['dfv_ch'] = np.std(df2_filtered_epochs.groupby(['subject','channel'])['df_ep'])
# 



In [148]:
%%time
%%capture
dominant = []
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2.5*srate), int(1.25*srate), int(2.5*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'california'
            features['subject'] = subject
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df = pd.DataFrame(dict_list)
            df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
df['dfv_ch'] = df['dfv_ch'].fillna(0)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/california/derivatives/sovaharmony/df_california_5s_no_overlapping.csv', index=False)
print('CSV CREATED')


Wall time: 12min 23s


##### Finland ####

In [18]:
finland={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FZ', 'CZ', 'PZ', 'IZ', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'FT9', 'FT10', 'TP9', 'TP10', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FPZ', 'CPZ', 'POZ', 'OZ'],
    'path':"D:/EEGs/PhD_datasets/BIDS/finland/"
}

#Path of the BIDS folder
bids_root = "D:/EEGs/PhD_datasets/BIDS/finland/"
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=finland #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

38 ['D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finland\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\finla

In [ ]:
%%capture
%%time
dominant = []
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    print('BANDPASS FILTER 4-15 HZ & DOWNSAMPLE DONE')
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2.5*srate), int(1.25*srate), int(2.5*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'finland'
            features['subject'] = subject
            features['channel'] = ch_label
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df = pd.DataFrame(dict_list)
            df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
df['dfv_ch'] = df['dfv_ch'].fillna(0)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/finland/derivatives/sovaharmony/df_finland_5s_no_overlapping.csv', index=False)
print('CSV CREATED')

In [19]:
%%capture
%%time
#eegs = ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif']
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(18)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c    
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = 500
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'finland'
        features['subject'] = subject
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
df = pd.DataFrame(dict_list)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/finland/derivatives/sovaharmony/df_finland_5s_averaged.csv', index=False)

# df2 = pd.read_csv('D:/escritorio/FDA/medellin/df_medellin_5s_no_overlapping.csv')
# df2_filtered_epochs = df2[df2['epochs'] <= 28]
# df2_filtered_epochs['dfv_ch'] = np.std(df2_filtered_epochs.groupby(['subject','channel'])['df_ep'])
# 


# df.to_csv('D:/EEGs/PhD_datasets/BIDS/finland/derivatives/sovaharmony/df_finland_5s_averaged.csv', index=False)


In [258]:
    dict_list = []
    for eeg_file in eegs: #read and preload dataset
        subject_info = layout.parse_file_entities(eeg_file)
        session = subject_info.get('session')
        subject = subject_info.get('subject')
        bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

        data_preload = {
            "preload": True
        }
        epochs = mne.read_epochs(eeg_file, preload = True)
        standardize(epochs) #standardize ch_names
        epochs = epochs.filter(l_freq=1, h_freq=30) #bandpassing 1-30Hz
        downsample = 500 #downsampling to 500Hz
        epochs.resample(downsample)
        sf = downsample
        print('BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE')
        subject_info = layout.parse_file_entities(eeg_file)
        session = subject_info.get('session')
        subject = subject_info.get('subject')
        nepochs,nchannels,npoints = epochs._data.shape
        correct_channels = epochs.info['ch_names']
        epochs.reorder_channels(correct_channels)
        channels = epochs.info['ch_names']
        for ch,ch_label in enumerate(channels):
            for ep in range(nepochs):
                features = {}
                features['center'] = 'finland'
                features['subject'] = subject
                features['channel'] = ch_label
                features['epoch'] = ep
                features['df_ep'] = peak_freq(epochs.get_data()[ep,ch,:], sf, [4, 15],'multitaper')
                dict_list.append(features)
                df = pd.DataFrame(dict_list)
                df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
    df['dfv_ch'] = df['dfv_ch'].fillna(0)
print('CSV CREATED')
df.to_csv('D:/EEGs/PhD_datasets/BIDS/finland/derivatives/sovaharmony/df_mt_finland_5s_no_overlapping.csv', index=False)



Reading D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-001\eeg\sub-001_task-rest_desc-reject_eeg.fif ...


C:\Users\acer\AppData\Local\Temp/ipykernel_3688/2405570582.py:11: RuntimeWarning: This filename (D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-001\eeg\sub-001_task-rest_desc-reject_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(eeg_file, preload = True)


Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
25 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE
Reading D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-002\eeg\sub-002_task-rest_desc-reject_eeg.fif ...


C:\Users\acer\AppData\Local\Temp/ipykernel_3688/2405570582.py:11: RuntimeWarning: This filename (D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-002\eeg\sub-002_task-rest_desc-reject_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(eeg_file, preload = True)


Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
25 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE
Reading D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-003\eeg\sub-003_task-rest_desc-reject_eeg.fif ...


...

Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
27 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE
Reading D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-038\eeg\sub-038_task-rest_desc-reject_eeg.fif ...


C:\Users\acer\AppData\Local\Temp/ipykernel_3688/2405570582.py:11: RuntimeWarning: This filename (D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-038\eeg\sub-038_task-rest_desc-reject_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(eeg_file, preload = True)


Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
24 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE
Reading D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-039\eeg\sub-039_task-rest_desc-reject_eeg.fif ...


C:\Users\acer\AppData\Local\Temp/ipykernel_3688/2405570582.py:11: RuntimeWarning: This filename (D:\EEGs\PhD_datasets\BIDS\finland\derivatives\sovaharmony\sub-039\eeg\sub-039_task-rest_desc-reject_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(eeg_file, preload = True)


Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
24 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 sec)

BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE
CSV CREATED


#### Iowa ####

In [21]:
iowa={
'layout':{'extension':'.fif', 'suffix':'eeg', 'return_type':'filename'},
    'ch_names':['T8', 'CP3', 'T7', 'C4', 'C5', 'FT8', 'F6', 'FC5', 'FC3', 'C2', 'TP9', 'F7', 'C6', 'CP6', 'F2', 'TP7', 'PO7', 'FCZ', 'O2', 'P2', 'CZ', 'F1', 'P6', 'AF8', 'C3', 'AF4', 'CPZ', 'F5', 'CP1', 'O1', 'P8', 'FC6', 'FC2', 'P4', 'F8', 'FP2', 'AF3', 'POZ', 'FT10', 'P1', 'FC1', 'TP10', 'FP1', 'AF7', 'PO8', 'TP8', 'F4', 'FZ', 'AFZ', 'OZ', 'CP4', 'P3', 'FT7', 'CP5', 'P5', 'P7', 'F3', 'FC4', 'CP2', 'C1'],
    'path':"D:/EEGs/PhD_datasets/BIDS/iowa/"
}

#Path of the BIDS folder
bids_root = "D:/EEGs/PhD_datasets/BIDS/iowa/"
#Seleccionar solo EEG
datatype = 'eeg'
suffix = 'eeg'

#Tarea
task = 'rest' 

DATASET=iowa #DEFINE DATASET

layoutd = DATASET.get('layout', None)

layout = bids.BIDSLayout(DATASET.get('path', None), derivatives=True)
eegs = layout.get(**layoutd)
eegs = [k for k in eegs if 'desc-reject' in k]

print(len(eegs), eegs)

28 ['D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-002\\eeg\\sub-002_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-003\\eeg\\sub-003_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-004\\eeg\\sub-004_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-005\\eeg\\sub-005_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-006\\eeg\\sub-006_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-007\\eeg\\sub-007_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmony\\sub-008\\eeg\\sub-008_task-rest_desc-reject_eeg.fif', 'D:\\EEGs\\PhD_datasets\\BIDS\\iowa\\derivatives\\sovaharmon

In [21]:
%%capture
%%time
#%%capture
dominant = []
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            srate = downsample #This gets the sampling rate of the EEGLAB object
            nfft, noverlap, nperseg = int(2.5*srate), int(1.25*srate), int(2.5*srate)
            freqs, psd = signal.welch(epochs._data[ep,ch,:], fs=srate,  nfft=nfft, noverlap=noverlap, nperseg=nperseg, window='hamming', scaling='spectrum') #This calculates PSD Welch
            al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
            icf_ind = np.argmax(np.abs(al_psd))
            al_icf = al_freqs[icf_ind]
            dominant_freq = al_icf
            features = {}
            features['center'] = 'iowa'
            features['subject'] = subject
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = dominant_freq
            dict_list.append(features)
            df = pd.DataFrame(dict_list)
            df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
#    df['dfv_ch'] = abs(df.df_ep - df.groupby('channel')['df_ep'].shift(periods=1)) 
#    df['dfv_ch'] = df['dfv_ch'].fillna(0)
#    df2["dfv_mean"] = df["dfv_ch"].groupby(df["channel"]).mean()
#    df2 = pd.DataFrame(df2)
#    dom_freq = pd.merge(df, df2, on="channel")
#    dominant.append(dom_freq)
#    print('DOMINANT FREQUENCY DONE')
#dominant_frequency = pd.DataFrame(dominant)
df['dfv_ch'] = df['dfv_ch'].fillna(0)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/iowa/derivatives/sovaharmony/df_iowa_5s_no_overlapping.csv', index=False)
print('CSV CREATED')

In [22]:
%%capture
%%time
#eegs = ['D:\\EEGs\\PhD_datasets\\BIDS\\antioquia\\derivatives\\sovaharmony\\sub-001\\eeg\\sub-001_task-rest_desc-reject_eeg.fif']
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    epochs_c = mne.concatenate_epochs(epochs_list = [epochs[:][range(16)]], add_offset=False, on_mismatch='raise', verbose=None)
    epochs = epochs_c    
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        features = {}
        srate = 500
        n_fft, n_overlap, n_per_seg= int(2*srate), int(srate), int(2*srate)
        psd, freqs = mne.time_frequency.psd_array_welch(epochs.get_data()[:,ch,:], sfreq = srate, n_per_seg=n_per_seg, fmin=1, fmax=30, n_overlap = n_overlap, n_fft= n_fft, average='mean', verbose = False)
        psds_mean = psd.mean(0)
        psd = psds_mean
        al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
        icf_ind = np.argmax(np.abs(al_psd))
        al_icf = al_freqs[icf_ind]
        dominant_freq = al_icf
        features['center'] = 'iowa'
        features['subject'] = subject
        features['channel'] = ch_label
        features['df_ep'] = dominant_freq
        dict_list.append(features)
df = pd.DataFrame(dict_list)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/iowa/derivatives/sovaharmony/df_iowa_5s_averaged.csv', index=False)


#### MULTITAPER: ALPHA PEAK FREQUENCY 

In [2]:
def peak_freq(data, sf, band, method='multitaper', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      ONLY FOR WELCH METHOD. Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, fmin = low, fmax = high, bandwidth = 2, adaptive=True,
                                          normalization='full', low_bias=True, verbose=0) ## Taper lenght is equal to 2Hz 
                                                                        ##### as this "is the optimal value to observe the extended alpha band", in my opinion might be quite large peak bandwidth
    
    
    al_freqs, al_psd = trim_spectrum(freqs, psd, [4, 15])
    icf_ind = np.argmax(np.abs(al_psd))
    al_icf = al_freqs[icf_ind]    

    return al_icf

Enter the research center field in the features file, line of code number 28
Enter the file name and path for the features file, line of code number 37
The "band" parameter in the peak_freq function is defind as 4 - 14 Hz (extended alpha band)


In [145]:
dict_list = []
for eeg_file in eegs: #read and preload dataset
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    bids_path = BIDSPath(subject=subject, session=session, task=task, root=bids_root, datatype='eeg')

    data_preload = {
        "preload": True
    }
    epochs = mne.read_epochs(eeg_file, preload = True)
    standardize(epochs) #standardize ch_names
    epochs = epochs.filter(l_freq=1, h_freq=30) #bandpassing 1-30Hz
    downsample = 500 #downsampling to 500Hz
    epochs.resample(downsample)
    sf = downsample
    print('BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE')
    subject_info = layout.parse_file_entities(eeg_file)
    session = subject_info.get('session')
    subject = subject_info.get('subject')
    nepochs,nchannels,npoints = epochs._data.shape
    correct_channels = epochs.info['ch_names']
    epochs.reorder_channels(correct_channels)
    channels = epochs.info['ch_names']
    for ch,ch_label in enumerate(channels):
        for ep in range(nepochs):
            features = {}
            features['center'] = 'california'
            features['subject'] = subject
            features['channel'] = ch_label
            features['epoch'] = ep
            features['df_ep'] = peak_freq(epochs.get_data()[ep,ch,:], sf, [4, 14],'multitaper')
            dict_list.append(features)
            df = pd.DataFrame(dict_list)
            df['dfv_ch'] = abs(df.df_ep - df.groupby(['subject','channel'])['df_ep'].shift(periods=1))
df['dfv_ch'] = df['dfv_ch'].fillna(0)
df.to_csv('D:/EEGs/PhD_datasets/BIDS/FILEPATH_AND_FILENAME.csv', index=False)
print('CSV CREATED')


Reading D:\EEGs\PhD_datasets\BIDS\california\derivatives\sovaharmony\sub-hc1\ses-hc\eeg\sub-hc1_ses-hc_task-rest_desc-norm_eeg.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.05 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
31 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 15 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 15.00 Hz
- Upper transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 16.88 Hz)
- Filter length: 845 samples (1.650 sec)



C:\Users\acer\AppData\Local\Temp/ipykernel_3688/3618531071.py:11: RuntimeWarning: This filename (D:\EEGs\PhD_datasets\BIDS\california\derivatives\sovaharmony\sub-hc1\ses-hc\eeg\sub-hc1_ses-hc_task-rest_desc-norm_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(eeg_file, preload = True)


BANDPASS FILTER 1-30 HZ & DOWNSAMPLE DONE


#### CSA PLOTS TO COMPARE WITH DF ESTIMATIONS - WELCH - HAMMING WINDOW

In [129]:
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

#raw = mne.read_epochs("D:/EEGs/PhD_datasets/BIDS/california/derivatives/sovaharmony/sub-pd28/ses-on/eeg/sub-pd28_ses-on_task-rest_desc-norm_eeg.fif", preload=True)
srate = raw.info['sfreq'] #This gets the sampling rate of the EEGLAB object
nfft, noverlap, nperseg = int(2.5*srate), int(1.25*srate), int(2.5*srate)
f, periodograma = signal.welch(raw._data[:,:,:], fs=srate, nfft=n_fft, noverlap=n_overlap, nperseg=n_per_seg, window='hamming', scaling='spectrum') #This calculates PSD Welch

In [119]:
#raw = mne.read_epochs("D:/EEGs/PhD_datasets/BIDS/california/derivatives/sovaharmony/sub-pd28/ses-on/eeg/sub-pd28_ses-on_task-rest_desc-norm_eeg.fif", preload=True)
srate = raw.info['sfreq'] #This gets the sampling rate of the EEGLAB object
n_fft, n_overlap, n_per_seg = int(2.5*srate), int(1.25*srate), int(2.5*srate)
periodograma, f = mne.time_frequency.psd_array_multitaper(raw._data[:,:,:], sfreq = srate, fmin=4, bandwidth= 1, fmax=15, adaptive=True, normalization='full', verbose=0)

In [144]:
df.head(60)

,center,subject,channel,epoch,df_ep,dfv_ch
0,medellin,071,C5,0,9.2,0.0
1,medellin,071,C5,1,12.8,3.6
2,medellin,071,C5,2,9.2,3.6
3,medellin,071,C5,3,12.0,2.8
4,medellin,071,C5,4,14.4,2.4
5,medellin,071,C5,5,13.6,0.8
6,medellin,071,C5,6,10.0,3.6
7,medellin,071,C5,7,8.8,1.2
8,medellin,071,C5,8,7.6,1.2
9,medellin,071,C5,9,13.6,6.0


In [49]:
raw.ch_names.index('C5')

0

In [140]:
channel_plot = 1 ### REPLACE HERE THE CHANNEL INDEX YOU WANNA PLOT ###
channel_name = raw.ch_names[channel_plot]

x = f
Nx = x.shape[0]

pot_tot = np.sum(periodograma,axis=-1)

num_epocas = periodograma.shape[0]
Nz = num_epocas
y1 = periodograma[:,channel_plot,:].T 

y = (y1/pot_tot[:,channel_plot])*100 


# create waterfall plot
fig = plt.figure(figsize=(10,15))
ax = fig.add_subplot(111)
for side in ['right', 'top', 'left']:
    ax.spines[side].set_visible(False)
    
# some usefull parameters
highest = np.max(y)
lowest = np.min(y)
delta = highest-lowest
t = np.sqrt(abs(delta))/4 # a tuning parameter for the offset of each dataset

for i in np.flip(range(Nz)):
    yi_ = y[:,i]       # the y data set
    yi = yi_ + i*t   # the shifted y data set used for plotting
    zindex = Nz-i # used to set zorder

    # fill with white from the (shifted) y data down to the lowest value
    # for good results, don't make the alpha (TRANSPARENCY) too low, otherwise you'll get confusing blending of lines
    ax.fill_between(x, lowest, yi, facecolor="white", alpha=0.8, zorder=zindex)

    # cut the data into segments that can be colored individually
    points = np.array([x, yi]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # Create a continuous norm to map from data points to colors
    norm = plt.Normalize(lowest, highest)
    lc = LineCollection(segments, cmap='plasma', norm=norm)
    
    # Set the values used for colormapping
    lc.set_array(yi_)
    lc.set_zorder(zindex)
    lc.set_linewidth(1)
    line = ax.add_collection(lc)
    
    # print text indicating epochs number
    delta_x = max(x)-min(x)
    if (i)%1==0:
        ax.text(min(x)-0.5, t*i, "%i"%i)


# set limits, as using LineCollection does not automatically set these
ax.set_xlim(3.8, 15.2)

psd_colorbar = fig.colorbar(line, ax=ax)
psd_colorbar.set_label( label = 'Relative PSD (%)', size = 18, weight='bold')


ax.set_title('Compressed Spectral Array ' + str(channel_name), size=22, fontweight="bold")
plt.yticks([])
ax.set_ylabel('Epoch', size=18, fontweight="bold")
ax.set_xlabel('Frequency', size=18, fontweight="bold")
ax.yaxis.set_label_coords(-0.05, 0.5)
ax.yaxis.set_ticks_position('none')